# WebUI 进行API调用

本地运行了：`bash webui.sh -f --api`

In [ ]:
import io
import os
import base64
import requests
import datetime
from PIL import Image

url = "http://0.0.0.0:7860"
payload_dict = {
    'Steps': '6', 
    'Sampler': 'DPM++ SDE Karras', 
    'Model': 'dreamshaperXL_sfwV2TurboDPMSDE', 'VAE': 'sdxl_vae.safetensors',
    'CFG scale': '2', 'Seed': '2567515004', 'Size': '1、16x912', 'Model hash': '6e718af03d', 
    'VAE hash': '63aeecb90f',  'Clip skip': '2', 'RNG': 'CPU'}
controlnet_0_dict = {
    'Module': 'canny',                    # 下面Model需要执行的预处理方式，也可以直接自己输入图像
    'Model':  'diffusers_xl_canny_full',  # canny图的权重
    'Weight': '0.6', 'Resize Mode': 'Crop and Resize', 'Processor Res': '512', 'Threshold A': '0.1', 'Threshold B': '0.1', 'Guidance Start': '0.0', 'Guidance End': '1.0', 'Pixel Perfect': 'True', 'Control Mode': 'ControlNet is more important'}

controlnet_1_dict = {
    'Module': 'depth_midas',              # 这个一般是指将图片转化的模型 比如说 depth_midas 可能就是直接使用其他深度处理模型去处理图片
    'Model':  'diffusers_xl_depth_full',  # 这个是 SDXL controlnet的模型权重
    'Weight': '0.6', 'Resize Mode': 'Crop and Resize', 'Processor Res': '512', 'Threshold A': '0.5', 'Threshold B': '0.5', 'Guidance Start': '0.0', 'Guidance End': '1.0', 'Pixel Perfect': 'True', 'Control Mode': 'My prompt is more important'}
# deepseek随机生成的 prompt测试
prompt = "(in the high-Contrast bright tones, white, gray, khaki and black color palette:1.3), <lora:Scandinavian_XL_40_1024_v1:0.3>, (beautiful detailed portrait:1.5), (photorealistic:1.4), (sharp focus:1.3), professional studio lighting, soft natural light, (elegant Scandinavian style clothing:1.4), minimalist fashion, (white and beige background:1.3), clean aesthetic, (HD 8K high details:1.5), (perfect eyes:1.4), (detailed skin texture:1.4), Nordic style portrait"
negative_prompt = "(worst quality:2), (low quality:2), lowres, bad anatomy, bad hands, (extra fingers:1.3), missing fingers, deformed face, (mutated hands and fingers:1.4), poorly drawn face, (mutation:1.3), ugly, blurry, (bad eyes:1.3), (cropped:1.2), (out of frame:1.2), text, error, jpeg artifacts, watermark, username, signature, (extra limbs:1.3), (fused fingers:1.4), (unclear eyes:1.3), bad-hands-5, EasyNegative"

def SDWebUI_image_generate(input_image, prompt, negative_prompt, width, height, payload_dict, controlnet_0_dict,controlnet_1_dict, controlnet_1_image=None):
    payload = {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "init_images": [input_image],
        'width': width,
        'height': height,
        "steps": payload_dict['Steps'],
        "cfg_scale": payload_dict["CFG scale"],
        "sampler_name": "DPM++ SDE Karras",
        "seed": -1,
    }
    override_settings = {}
    override_settings["sd_model_checkpoint"] = "dreamshaperXL_sfwV2TurboDPMSDE.safetensors"
    override_settings["sd_vae"] = "sdxl_vae.safetensors"
    override_settings["randn_source"] = payload_dict['RNG']
    override_settings["CLIP_stop_at_last_layers"] = int(payload_dict["Clip skip"])

    override_settings = {
        "override_settings": override_settings
    }
    payload.update(override_settings)

    controlnet_units = [
    {
        "enabled": True,
        "module": controlnet_0_dict['Module'],
        "model": controlnet_0_dict['Model'],
        "weight": float(controlnet_0_dict['Weight']),
        "threshold_a": float(controlnet_0_dict['Threshold A']),
        "threshold_b": float(controlnet_0_dict['Threshold B']),
        "guidance_start": float(controlnet_0_dict["Guidance Start"]),
        "guidance_end": float(controlnet_0_dict["Guidance End"]),
        "processor_res": int(controlnet_0_dict['Processor Res']),
        "lowvram": False,
        "hr_option": 'Both',
        "resize_mode": controlnet_0_dict['Resize Mode'],
        "control_mode": controlnet_0_dict['Control Mode'],
        "pixel_perfect": controlnet_0_dict['Pixel Perfect'],
        "input_image": input_image,
    },
    {
        "enabled": True,
        "input_image": controlnet_1_image if controlnet_1_image else input_image,
        "module": controlnet_1_dict['Module'],
        "model": controlnet_1_dict['Model'],
        "weight": float(controlnet_1_dict['Weight']),
        "guidance_start": float(controlnet_1_dict["Guidance Start"]),
        "guidance_end": float(controlnet_1_dict["Guidance End"]),
        "hr_option": 'Both',
        "processor_res": int(controlnet_1_dict['Processor Res']),
        "lowvram": False,
        "resize_mode": controlnet_1_dict['Resize Mode'],
        "control_mode": controlnet_1_dict['Control Mode'],
        "pixel_perfect": controlnet_1_dict['Pixel Perfect'],
    },
    ]
    # 添加 controlnet
    alwayson_scripts = {
        "ControlNet": {
            "args": controlnet_units,
        }
    }
    alwayson_scripts = {
        "alwayson_scripts": alwayson_scripts
    }
    payload.update(alwayson_scripts)
    response = requests.post(url=f'{url}/sdapi/v1/img2img', json=payload)
    json = response.json()

    images = json['images']
    if len(images) > 0:
        pic_time = datetime.datetime.now()
        save_path = f'./CodeWork/tmp/output{pic_time}.png'
        image = Image.open(io.BytesIO(base64.b64decode(images[0])))
        image.save(save_path)

def base64_img(image):
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getbuffer()
    return "data:image/png;base64," + base64.b64encode(img_byte_arr).decode('utf-8')

if __name__ == '__main__':
    raw_image_path = "./CodeWork/image/raw/xkdivpirs1y.jpg"
    depth_image_path = "./CodeWork/image/depth/depth.jpeg"

    ori_width, ori_height = Image.open(raw_image_path).size
    max_size = 1024
    if ori_width < ori_height:
        height = int(ori_height / ori_width * max_size)
        width = max_size
    else:
        width = int(ori_width / ori_height * max_size)
        height = max_size

    raw_image = Image.open(raw_image_path)
    depth_image = Image.open(depth_image_path)
    raw_image = raw_image.resize((width, height))
    depth_image = depth_image.resize((width, height))

    raw_image = base64_img(raw_image)
    depth_image = base64_img(depth_image)

    SDWebUI_image_generate(raw_image, prompt, negative_prompt, width, height, 
    payload_dict, controlnet_0_dict,controlnet_1_dict,
    controlnet_1_image= depth_image)